In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import glob
import cv2
import os
from keras.applications.densenet import DenseNet169
from keras.applications.mobilenet import MobileNet
from sklearn import preprocessing
from google.colab import drive
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score
import lightgbm as lgb

## loading the dataset

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
SIZE = 224
images = []
images_labels = [] 

### importing each image and set the current directory name as its label 

In [ ]:
ext = ['png', 'jpg', 'jpeg'] 

for directory_path in glob.glob("/content/gdrive/My Drive/seperated dataset/X-Ray Image DataSet/images/*"):
    splited = directory_path.split("/")
    label = splited[-1]
    print(label)
    for e in ext:
      for img_path in glob.glob(os.path.join(directory_path, "*." + e)):
        if label == 'Pneumonia':
          break
        else:
          #print(img_path)
           img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
           img = cv2.resize(img, (SIZE, SIZE))
           images.append(img)
           images_labels.append(label)

Pneumonia
No_findings
Covid-19


In [ ]:
images = np.array(images)
images_labels = np.array(images_labels)

In [ ]:
images.shape

(625, 224, 224, 3)

In [ ]:
np.unique(images_labels)

array(['Covid-19', 'No_findings'], dtype='<U11')

## Pre-processing

### label encoding

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(images_labels)
labels_encoded = le.transform(images_labels)

## Feature Extraction

### DenseNet169

In [ ]:
dense_model = DenseNet169(include_top=False, input_shape=(SIZE, SIZE, 3), pooling='avg')

51888128/51877672 [==============================] - 0s 0us/step


In [ ]:
dense_features = dense_model.predict(images)

In [ ]:
features = dense_features.reshape(dense_features.shape[0], -1)

In [ ]:
features.shape

(625, 1664)

### Mobilenet

In [ ]:
model_mobile = MobileNet(weights='imagenet',include_top=False, input_shape=(SIZE, SIZE, 3), pooling='avg')

In [ ]:
mobile_features = model_mobile.predict(images)

In [ ]:
features_2 = mobile_features.reshape(mobile_features.shape[0], -1)

In [ ]:
features_2.shape

(625, 1024)

## Feature Combination

In [ ]:
combined_features = np.hstack((dense_features, mobile_features))

## lightgbm classifier

In [ ]:
params = {'learning_rate':0.2, 'n_iterations': 500, 'max_depth': 7, 'num_leaves': 105, 'n_estimators': 300, 'min_child_samples': 40 }


In [ ]:
lgb_classifier = lgb.LGBMClassifier(**params)

## KFold cross validation

In [ ]:
accuracy_per_fold = {}

for i in range(0, 100):
  kf = KFold(n_splits=5, random_state=i, shuffle=True)
  accuracy = cross_val_score(lgb_classifier, combined_features, labels_encoded, cv=kf, scoring='accuracy')
  accuracy_per_fold[i] = np.mean(accuracy)

In [ ]:
max(accuracy_per_fold.values())

0.9856